# Upload Data Sets (Tech_Hub)

## Data Sets

In [49]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df1 = pd.read_excel("Metropolitan_Data_on_Tech.xlsx")
df1.rename(columns={'Employment Employment_percent_relative_standard_error Hourly_mean_wage Annual_mean_wage Wage_percent_relative_standard_error Hourly_10th_percentile_wage Hourly_25th_percentile_wage Hourly_median_wage Hourly_75th_percentile_wage Hourly_90th_percentile_wage Annual_10th_percentile_wage_(2) Annual_25th_percentile_wage_(2) Annual_median_wage_(2) Annual_75th_percentile_wage_(2) Annual_90th_percentile_wage_(2) Employment_per_1,000_jobs Location_Quotient': 'Employment_Total'}, inplace=True)
df1.rename(columns={'Employment_per_Thousand_Jobs': 'Employment_per_1k_jobs'}, inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Area_Name                                   538 non-null    object
 1   Employment_Total                            530 non-null    object
 2   Employment_percent_relative_standard_error  530 non-null    object
 3   Hourly_mean_wage                            530 non-null    object
 4   Annual_mean_wage                            530 non-null    object
 5   Wage_percent_relative_standard_error        530 non-null    object
 6   Hourly_10th_percentile_wage                 530 non-null    object
 7   Hourly_25th_percentile_wage                 530 non-null    object
 8   Hourly_median_wage                          530 non-null    object
 9   Hourly_75th_percentile_wage                 530 non-null    object
 10  Hourly_90th_percentile_wag

In [50]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df2 = pd.read_excel("Tech_Job_Gains.xlsx")

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    51 non-null     object
 1   2018    51 non-null     int64 
 2   2019    51 non-null     int64 
 3   2020    51 non-null     int64 
 4   2021    51 non-null     int64 
 5   2022    51 non-null     int64 
 6   2023    51 non-null     int64 
 7   2024    51 non-null     int64 
dtypes: int64(7), object(1)
memory usage: 3.3+ KB


In [309]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df3 = pd.read_excel("Venture_Capitol.xlsx")

df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Location               150 non-null    object 
 1   Venture_Capital_Score  150 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.5+ KB


In [32]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df11 = pd.read_csv("top_100_us_tech_cities.csv")

df11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   100 non-null    int64 
 1   City                   100 non-null    object
 2   Number_of_Tech_Events  100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


In [64]:
import pandas as pd

# Load the Excel files into Pandas DataFrames
df12 = pd.read_csv("top_100_us_tech_layoffs_per_capita.csv")

df12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Location            100 non-null    object 
 1   Population          100 non-null    int64  
 2   Number_of_Layoffs   100 non-null    int64  
 3   Layoffs_per_Capita  100 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.2+ KB


## Merging the Data Sets

In [91]:
import pandas as pd
import sqlite3

# Load the Excel files into Pandas DataFrames
df1 = pd.read_excel("Metropolitan_Data_on_Tech.xlsx")
df2 = pd.read_excel("Tech_Job_Gains.xlsx")
df11 = pd.read_csv("top_100_us_tech_cities.csv")
df12 = pd.read_csv("top_100_us_tech_layoffs_per_capita.csv")



# Adjust specific city names to match between datasets
df2['City'] = df2['City'].replace('New York City', 'New York')
df1['Area_Name'] = df1['Area_Name'].replace('Washington-Arlington-Alexandria, DC-VA-MD-WV (00-47900)', 
                                            'Washington DC-Arlington-Alexandria, DC-VA-MD-WV (00-47900)')

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Load DataFrames into SQLite tables
df1.to_sql("Metro_Data", conn, if_exists="replace", index=False)
df2.to_sql("Tech_Gains", conn, if_exists="replace", index=False)
df11.to_sql("Tech_Events", conn, if_exists="replace", index=False)
df12.to_sql("Tech_Layoffs", conn, if_exists="replace", index=False)


# SQL Query to merge all datasets
query = """
SELECT DISTINCT *
FROM Metro_Data 
JOIN Tech_Gains ON Metro_Data.Area_Name LIKE '%' || Tech_Gains.City || '%'
AND Metro_Data.Area_Name NOT LIKE '%nonmetropolitan%'
LEFT JOIN Tech_Layoffs ON Tech_Layoffs.Location LIKE '%' || Tech_Gains.City || '%'
LEFT JOIN Tech_Events ON Tech_Events.Spot LIKE '%' || Tech_Gains.City || '%'
GROUP BY Metro_Data.Area_Name

"""

# Load the merged result back into a Pandas DataFrame
merged_df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Save or analyze the merged data
merged_df.to_excel("merged_output.xlsx", index=False)  # Save as Excel
merged_df  # Display first few rows



,Area_Name,"Employment Employment_percent_relative_standard_error Hourly_mean_wage Annual_mean_wage Wage_percent_relative_standard_error Hourly_10th_percentile_wage Hourly_25th_percentile_wage Hourly_median_wage Hourly_75th_percentile_wage Hourly_90th_percentile_wage Annual_10th_percentile_wage_(2) Annual_25th_percentile_wage_(2) Annual_median_wage_(2) Annual_75th_percentile_wage_(2) Annual_90th_percentile_wage_(2) Employment_per_1,000_jobs Location_Quotient",Employment_percent_relative_standard_error,Hourly_mean_wage,Annual_mean_wage,Wage_percent_relative_standard_error,Hourly_10th_percentile_wage,Hourly_25th_percentile_wage,Hourly_median_wage,Hourly_75th_percentile_wage,...,2021,2022,2023,2024,Location,Population,Number_of_Layoffs,Layoffs_per_Capita,Spot,Number_of_Tech_Events
0,"Albuquerque, NM (00-10740)",11730,3.29999995231628,46.7,97140,2.40000009536743,20.9,30.12,42.53,61.91,...,21778,22898,23276,23596,"Albuquerque, NM",561188.0,3464.0,0.006173,"Albuquerque, NM",98
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",120180,1.5,52.5,109210,0.699999988079071,26.05,36.34,50.12,65.4,...,217589,227821,229584,235268,"Atlanta, GA",5234502.0,15838.0,0.003026,"Atlanta, GA",2937
2,"Austin-Round Rock, TX (00-12420)",81150,1.39999997615814,53.6,111490,2.59999990463257,24.43,35.07,50.64,67.85,...,157572,176248,180507,190186,"Austin, TX",4391747.0,1353.0,0.000308,"Austin, TX",996
3,"Baltimore-Columbia-Towson, MD (00-12580)",64180,1.79999995231628,61.76,128450,3.40000009536743,30.93,42.44,60.23,77.72,...,104078,104423,104838,106852,"Baltimore, MD",5370385.0,5677.0,0.001057,"Baltimore, MD",1857
4,"Boise City, ID (00-14260)",10580,2.70000004768372,53.76,111810,7.80000019073486,22.64,30.15,41.56,60.36,...,24048,25387,25749,26592,"Boise, ID",7963697.0,17464.0,0.002193,"Boise, ID",951
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",136640,1.29999995231628,61.92,128790,0.899999976158142,33.19,43.83,60.66,77.69,...,266780,270775,268934,274023,"Boston, MA",736976.0,17458.0,0.023689,"Boston, MA",1312
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",11770,1.39999997615814,46.88,97510,0.800000011920929,23.48,31.71,43.26,59,...,22815,23981,24550,25255,"Buffalo, NY",8566999.0,1581.0,0.000185,"Buffalo, NY",1731
7,"Charleston, WV (00-16620)",2540,4,36.92,76790,1.70000004768372,19.45,24.23,32.45,45.94,...,17759,19162,19588,20468,"Charleston, SC",2033509.0,13275.0,0.006528,"Charleston, SC",435
8,"Charleston-North Charleston, SC (00-16700)",11770,2.79999995231628,49.1,102120,1.29999995231628,24.33,33.72,46.41,60.59,...,17759,19162,19588,20468,"Charleston, SC",2033509.0,13275.0,0.006528,"Charleston, SC",435
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",60370,0.899999976158142,55.78,116020,0.5,28.47,39.38,55.06,68.69,...,85263,89246,90859,93806,"Charlotte, NC",6374167.0,15702.0,0.002463,"Charlotte, NC",2241


### 1. Add ***Tech Job Growth Column*** & ***Entry Level Wage Average (10th and 25th Percentile)***

In [93]:
import pandas as pd
Merged_Output = pd.read_excel("merged_output.xlsx")
Merged_Output.rename(columns={'Employment Employment_percent_relative_standard_error Hourly_mean_wage Annual_mean_wage Wage_percent_relative_standard_error Hourly_10th_percentile_wage Hourly_25th_percentile_wage Hourly_median_wage Hourly_75th_percentile_wage Hourly_90th_percentile_wage Annual_10th_percentile_wage_(2) Annual_25th_percentile_wage_(2) Annual_median_wage_(2) Annual_75th_percentile_wage_(2) Annual_90th_percentile_wage_(2) Employment_per_1,000_jobs Location_Quotient': 'Employment_Total'}, inplace=True)
Merged_Output.rename(columns={'Employment_per_Thousand_Jobs': 'Employment_per_1k_jobs'}, inplace=True)



final_value = Merged_Output['2023']
intial_value = Merged_Output['2018']
years = 6

Merged_Output['Average_7_Year_Growth_Rate'] = (((final_value / intial_value) ** (1 / years) - 1) * 100).round(3)
Merged_Output['Entry_Level_Tech_Wage'] = (Merged_Output['Annual_10th_percentile_wage'] + Merged_Output['Annual_25th_percentile_wage'])/2

# Optionally, you can save the result to a new Excel file
Merged_Output.to_excel("updated_output.xlsx", index=False)

df4 = pd.read_excel('updated_output.xlsx')
df4


,Area_Name,Employment_Total,Employment_percent_relative_standard_error,Hourly_mean_wage,Annual_mean_wage,Wage_percent_relative_standard_error,Hourly_10th_percentile_wage,Hourly_25th_percentile_wage,Hourly_median_wage,Hourly_75th_percentile_wage,...,2023,2024,Location,Population,Number_of_Layoffs,Layoffs_per_Capita,Spot,Number_of_Tech_Events,Average_7_Year_Growth_Rate,Entry_Level_Tech_Wage
0,"Albuquerque, NM (00-10740)",11730,3.3,46.70,97140,2.4,20.90,30.12,42.53,61.91,...,23276,23596,"Albuquerque, NM",561188.0,3464.0,0.006173,"Albuquerque, NM",98,2.111,53065
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",120180,1.5,52.50,109210,0.7,26.05,36.34,50.12,65.40,...,229584,235268,"Atlanta, GA",5234502.0,15838.0,0.003026,"Atlanta, GA",2937,1.500,64890
2,"Austin-Round Rock, TX (00-12420)",81150,1.4,53.60,111490,2.6,24.43,35.07,50.64,67.85,...,180507,190186,"Austin, TX",4391747.0,1353.0,0.000308,"Austin, TX",996,4.914,61875
3,"Baltimore-Columbia-Towson, MD (00-12580)",64180,1.8,61.76,128450,3.4,30.93,42.44,60.23,77.72,...,104838,106852,"Baltimore, MD",5370385.0,5677.0,0.001057,"Baltimore, MD",1857,0.653,76300
4,"Boise City, ID (00-14260)",10580,2.7,53.76,111810,7.8,22.64,30.15,41.56,60.36,...,25749,26592,"Boise, ID",7963697.0,17464.0,0.002193,"Boise, ID",951,1.619,54905
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",136640,1.3,61.92,128790,0.9,33.19,43.83,60.66,77.69,...,268934,274023,"Boston, MA",736976.0,17458.0,0.023689,"Boston, MA",1312,0.426,80095
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",11770,1.4,46.88,97510,0.8,23.48,31.71,43.26,59.00,...,24550,25255,"Buffalo, NY",8566999.0,1581.0,0.000185,"Buffalo, NY",1731,0.236,57395
7,"Charleston, WV (00-16620)",2540,4.0,36.92,76790,1.7,19.45,24.23,32.45,45.94,...,19588,20468,"Charleston, SC",2033509.0,13275.0,0.006528,"Charleston, SC",435,1.748,45425
8,"Charleston-North Charleston, SC (00-16700)",11770,2.8,49.10,102120,1.3,24.33,33.72,46.41,60.59,...,19588,20468,"Charleston, SC",2033509.0,13275.0,0.006528,"Charleston, SC",435,1.748,60380
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",60370,0.9,55.78,116020,0.5,28.47,39.38,55.06,68.69,...,90859,93806,"Charlotte, NC",6374167.0,15702.0,0.002463,"Charlotte, NC",2241,2.272,70565


### Final Clean

In [99]:
import pandas as pd

# Load the initial data from the Excel file
df5 = pd.read_excel("updated_output.xlsx")

# Drop unnecessary columns from df5 as per your previous step

# Define the weights for each column in the calculation of the "Entry Level Tech Friendly Index"
weights = {
    'Employment_per_1k_jobs': 0.25,
    'Location_Quotient': 0.20,
    'Average_7_Year_Growth_Rate': 0.30,
    'Entry_Level_Tech_Wage': 0.15,
    'Number_of_Tech_Events': 0.10,
    'Layoffs_per_Capita': 0.10
}

# Calculate the "Entry Level Tech Friendly Index" by combining the weighted columns
df5['Tech_Friendly_Index'] = (
    df5['Employment_per_1k_jobs'] * weights['Employment_per_1k_jobs'] +
    df5['Location_Quotient'] * weights['Location_Quotient'] +
    df5['Average_7_Year_Growth_Rate'] * weights['Average_7_Year_Growth_Rate'] +
    df5['Entry_Level_Tech_Wage'] * weights['Entry_Level_Tech_Wage'] +
    df5['Number_of_Tech_Events'] * weights['Number_of_Tech_Events'] -
    df5['Layoffs_per_Capita'] * weights['Layoffs_per_Capita']


)

# Save the updated DataFrame with the new index back to an Excel file
df5.to_excel("Tech_Hub.xlsx", index=False)

# Optional: Print the updated DataFrame to check the new column
df5



,Area_Name,Employment_Total,Employment_percent_relative_standard_error,Hourly_mean_wage,Annual_mean_wage,Wage_percent_relative_standard_error,Hourly_10th_percentile_wage,Hourly_25th_percentile_wage,Hourly_median_wage,Hourly_75th_percentile_wage,...,2024,Location,Population,Number_of_Layoffs,Layoffs_per_Capita,Spot,Number_of_Tech_Events,Average_7_Year_Growth_Rate,Entry_Level_Tech_Wage,Tech_Friendly_Index
0,"Albuquerque, NM (00-10740)",11730,3.3,46.70,97140,2.4,20.90,30.12,42.53,61.91,...,23596,"Albuquerque, NM",561188.0,3464.0,0.006173,"Albuquerque, NM",98,2.111,53065,7977.764683
1,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",120180,1.5,52.50,109210,0.7,26.05,36.34,50.12,65.40,...,235268,"Atlanta, GA",5234502.0,15838.0,0.003026,"Atlanta, GA",2937,1.500,64890,10038.562698
2,"Austin-Round Rock, TX (00-12420)",81150,1.4,53.60,111490,2.6,24.43,35.07,50.64,67.85,...,190186,"Austin, TX",4391747.0,1353.0,0.000308,"Austin, TX",996,4.914,61875,9399.131668
3,"Baltimore-Columbia-Towson, MD (00-12580)",64180,1.8,61.76,128450,3.4,30.93,42.44,60.23,77.72,...,106852,"Baltimore, MD",5370385.0,5677.0,0.001057,"Baltimore, MD",1857,0.653,76300,11643.484794
4,"Boise City, ID (00-14260)",10580,2.7,53.76,111810,7.8,22.64,30.15,41.56,60.36,...,26592,"Boise, ID",7963697.0,17464.0,0.002193,"Boise, ID",951,1.619,54905,8338.603981
5,"Boston-Cambridge-Nashua, MA-NH (00-71650)",136640,1.3,61.92,128790,0.9,33.19,43.83,60.66,77.69,...,274023,"Boston, MA",736976.0,17458.0,0.023689,"Boston, MA",1312,0.426,80095,12158.233431
6,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",11770,1.4,46.88,97510,0.8,23.48,31.71,43.26,59.00,...,25255,"Buffalo, NY",8566999.0,1581.0,0.000185,"Buffalo, NY",1731,0.236,57395,8788.135782
7,"Charleston, WV (00-16620)",2540,4.0,36.92,76790,1.7,19.45,24.23,32.45,45.94,...,20468,"Charleston, SC",2033509.0,13275.0,0.006528,"Charleston, SC",435,1.748,45425,6864.100497
8,"Charleston-North Charleston, SC (00-16700)",11770,2.8,49.10,102120,1.3,24.33,33.72,46.41,60.59,...,20468,"Charleston, SC",2033509.0,13275.0,0.006528,"Charleston, SC",435,1.748,60380,9109.058747
9,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",60370,0.9,55.78,116020,0.5,28.47,39.38,55.06,68.69,...,93806,"Charlotte, NC",6374167.0,15702.0,0.002463,"Charlotte, NC",2241,2.272,70565,10821.310853


### Calculate "Tech Friendly Index"

# Confortable Living Index

In [313]:
import pandas as pd 

Entry_Level_Wage= pd.read_excel("Final_Tech_Data.xlsx")
Entry_Level_Wage.drop(['Employment_Total', 'Employment_per_1k_jobs', 'Location_Quotient', 'Average_7_Year_Growth_Rate'], axis=1, inplace=True)
Entry_Level_Wage.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Area_Name              56 non-null     object
 1   Entry_Level_Tech_Wage  56 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [109]:
import pandas as pd

COL = pd.read_csv("advisorsmith_cost_of_living_index.csv")
COL

,City,State,Cost of Living Index
0,Abilene,TX,89.1
1,Adrian,MI,90.5
2,Akron,OH,89.4
3,Alamogordo,NM,85.8
4,Albany,GA,87.3
...,...,...,...
505,Wheeling,WV,84.1
506,New London,CT,105.9
507,Daphne,AL,96.6
508,Victoria,TX,89.5


In [16]:
import pandas as pd

TB = pd.read_excel("Tax_Burden.xlsx")
TB.head()

,State,Sales_Tax,Income_Tax,Property_Tax,License_Tax,Total_Tax
0,AL,4936,3870.0,337.0,494,9637.0
1,AK,256,228.0,128.0,147,759.0
2,AZ,8246,4452.0,883.0,459,14040.0
3,AR,4410,3141.0,1088.0,384,9023.0
4,CA,52347,86937.0,2316.0,9422,151022.0


### Calculating Financial Hub

In [133]:
import pandas as pd
import sqlite3

# Load the datasets
Entry_Level_Wage = pd.read_excel("Final_Tech_Data.xlsx")
COL = pd.read_csv("advisorsmith_cost_of_living_index.csv")
TB = pd.read_excel("Tax_Burden.xlsx")

# Drop unnecessary columns from Entry_Level_Wage
Entry_Level_Wage.drop(['Employment_Total', 'Employment_per_1k_jobs', 'Location_Quotient', 'Average_7_Year_Growth_Rate'], axis=1, inplace=True)

# Standardize column names for easier SQL usage
COL.rename(columns={"Cost of Living Index": "Cost_Living"}, inplace=True)

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Load the DataFrames into SQLite
Entry_Level_Wage.to_sql("Tech_Wages", conn, if_exists="replace", index=False)
COL.to_sql("Cost_Living", conn, if_exists="replace", index=False)
TB.to_sql("Tax_Burden", conn, if_exists="replace", index=False)


# SQL query to calculate the ratios
query = """
SELECT Cost_Living.City, Tech_Wages.Area_Name, Tax_Burden.State,
((Tech_Wages.Entry_Level_Tech_Wage) / (Cost_Living.Cost_Living * 100)) - (Tax_Burden.Total_Tax / Tech_Wages.Entry_Level_Tech_Wage)
 AS Financial_Comfort_Index
FROM Tech_Wages
JOIN Cost_Living ON Tech_Wages.Area_Name LIKE '%' || Cost_Living.City || '%'
JOIN Tax_Burden ON Tech_Wages.Area_Name LIKE '%, %' || Tax_Burden.State || '%'
GROUP BY Tech_Wages.Area_Name;

"""

# Execute query and store the result in a DataFrame
result_df = pd.read_sql(query, conn)

# Close the SQLite connection
conn.close()

# Display results
result_df.head()

# Optionally, save the result to an Excel file
result_df.to_excel("Financial_Hub.xlsx", index=False)
result_df

,City,Area_Name,State,Financial_Comfort_Index
0,Albuquerque,"Albuquerque, NM (00-10740)",NM,5.617681
1,Atlanta,"Atlanta-Sandy Springs-Roswell, GA (00-12060)",GA,6.165646
2,Austin,"Austin-Round Rock, TX (00-12420)",TX,4.978857
3,Baltimore,"Baltimore-Columbia-Towson, MD (00-12580)",MD,6.878705
4,Boise City,"Boise City, ID (00-14260)",ID,5.401789
5,Boston,"Boston-Cambridge-Nashua, MA-NH (00-71650)",MA,5.713098
6,Buffalo,"Buffalo-Cheektowaga-Niagara Falls, NY (00-15380)",NY,4.600187
7,Charleston,"Charleston, WV (00-16620)",WV,5.306622
8,Charleston,"Charleston-North Charleston, SC (00-16700)",SC,7.038316
9,Charlotte,"Charlotte-Concord-Gastonia, NC-SC (00-16740)",NC,6.853667


## Combine Both

---

# Machine Learning Begins


### 1. Loading Data Sets & Defining Targets

In [134]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


# Load datasets
tech_data = pd.read_excel("Tech_Hub.xlsx")
affordability_data = pd.read_excel("Financial_Hub.xlsx")

# Merge datasets on 'Area_Name'
df = pd.merge(tech_data, affordability_data, on='Area_Name', how='inner')

# Define features (X) and targets (Y1: Tech Hub, Y2: Affordability)
X = df.drop(columns=['Area_Name'])
Y1 = df['Tech_Friendly_Index']  # Binary classification (Yes/No)
Y2 = df['Financial_Comfort_Index']  # Binary classification (Yes/No)

### 2. Split data into training and testing sets

In [135]:
# Standardize the features
df_clean = df.select_dtypes(include=['number'])
X = df_clean.drop(columns=['Tech_Friendly_Index', 'Financial_Comfort_Index'])  # Drop target columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, Y1_train, Y1_test = train_test_split(X_scaled, Y1, test_size=0.2, random_state=42)
X_train, X_test, Y2_train, Y2_test = train_test_split(X_scaled, Y2, test_size=0.2, random_state=42)